In [ ]:
#!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/test/images
import argparse
import os
import platform
import sys
from pathlib import Path

import torch

FILE = Path('__file__').resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode


In [ ]:
import argparse
import os
import platform
import sys
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import torch
from utils.general import (CONFIG_DIR, FONT, LOGGER, check_font, check_requirements, clip_boxes, increment_path,
                           is_ascii, xywh2xyxy, xyxy2xywh)
FILE = Path('__file__').resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode
#from deep_sort.tracker import Tracker
#!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/test/images
import tracker
from collections import deque
from sklearn.utils.linear_assignment_ import linear_assignment
#--weights runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/test/images
@smart_inference_mode()
def run(
        weights='pre-trained model.pt',  # model path or triton URL
        source='test/input.mp4',  # file/dir/URL/glob/screen/0(webcam)
        data=ROOT / 'data/coco128.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.4,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        reid_weights= 'osnet_x0_25_msmt17.pt',  # model.pt path,
        tracking_method='strongsort',
        tracking_config=None,
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project=ROOT / 'runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
        vid_stride=1,  # video frame-rate stride
):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.streams') or (is_url and not is_file)
    screenshot = source.lower().startswith('screen')
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    bs = 1  # batch_size
    if webcam:
        view_img = check_imshow(warn=True)
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
        bs = len(dataset)
    elif screenshot:
        dataset = LoadScreenshots(source, img_size=imgsz, stride=stride, auto=pt)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt, vid_stride=vid_stride)
    vid_path, vid_writer = [None] * bs, [None] * bs
    
    #for i in range(bs):
        #tracker = create_tracker(tracking_method, tracking_config, reid_weights, device, half)
        #tracker_list.append(tracker, )
        #if hasattr(tracker_list[i], 'model'):
            #if hasattr(tracker_list[i].model, 'warmup'):
                #tracker_list[i].model.warmup()
    #outputs = [None] * nr_sources

    # Run inference
    model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], (Profile(), Profile(), Profile())
    for path, im, im0s, vid_cap, s in dataset:
        print(vid_cap)
        with dt[0]:
            im = torch.from_numpy(im).to(model.device)
            im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
            im /= 255  # 0 - 255 to 0.0 - 1.0
            if len(im.shape) == 3:
                im = im[None]  # expand for batch dim

        # Inference
        with dt[1]:
            visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
            pred = model(im, augment=augment, visualize=visualize)

        # NMS
        with dt[2]:
            pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)
        
        # Process predictions
        for i, det in enumerate(pred):  # per imag
            
            #os.mkdir('temp/')
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
                print("frame",frame)
            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, 5].unique():
                    n = (det[:, 5] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                j=0
                for *xyxy, conf, cls in reversed(det):
                    x_box=[]
                    tracker_list=[]
                    min_hits =1  
                    max_age = 4
                    track_id_list= deque(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'])
                    for trk in tracker_list:
                        trk.kalman_filter(trk.box)
                        x_box.append(trk.box)
                    print("tracking" ,x_box)
                    detector=det
                    matched,unmatched_dets,unmatched_trks \
                            =assign_detections_to_trackers(x_box,detector,iou_thrd=0.2)
                    print('Detection: ', detector)
                    print('x_box: ', x_box)
                    print('matched:', matched)
                    print('unmatched_det:', unmatched_dets)
                    print('unmatched_trks:', unmatched_trks)
                    if matched.size >0:
                        for trk_idx, det_idx in matched:
                            z = detector[det_idx]
                            z = np.expand_dims(z, axis=0).T
                            #img=helpers.add_class(img,c[det_idx],z)
                            tmp_trk= tracker_list[trk_idx]

                            tmp_trk.kalman_filter(z)
                            xx = tmp_trk.x_state.T[0].tolist()
                            xx =[xx[0], xx[2], xx[4], xx[6]]
                            x_box[trk_idx] = xx
                            tmp_trk.box =xx
                            tmp_trk.hits += 1
                            tmp_trk.no_losses = 0
                    # Deal with unmatched detections      
                    if len(unmatched_dets)>0:
                        for idx in unmatched_dets:
                            z = detector[idx]
                            z = np.expand_dims(z, axis=0).T
                            tmp_trk = tracker.Tracker() # Create a new tracker
                            x = np.array([[z[0], 0, z[1], 0, z[2], 0, z[3], 0]]).T
                            tmp_trk.x_state = x
                            tmp_trk.predict_only()
                            xx = tmp_trk.x_state
                            xx = xx.T[0].tolist()
                            xx =[xx[0], xx[2], xx[4], xx[6]]
                            tmp_trk.box = xx
                            tmp_trk.id = track_id_list.popleft() # assign an ID for the tracker

                            print(tmp_trk.id)
                            tracker_list.append(tmp_trk)
                            x_box.append(xx)
                    if len(unmatched_trks)>0:
                        for trk_idx in unmatched_trks:
                            tmp_trk = tracker_list[trk_idx]
                            tmp_trk.no_losses += 1
                            tmp_trk.predict_only()
                            xx = tmp_trk.x_state
                            xx = xx.T[0].tolist()
                            xx =[xx[0], xx[2], xx[4], xx[6]]
                            tmp_trk.box =xx
                            x_box[trk_idx] = xx


                    # The list of tracks to be annotated  
                    good_tracker_list =[]
                    for trk in tracker_list:
                        if ((trk.hits >= min_hits) and (trk.no_losses <=max_age)):
                            good_tracker_list.append(trk)
                            print('trackerlist: ',good_tracker_list)
                            c=trk.label
                            x_cv2 = trk.box
                            track_id=trk.id
                    deleted_tracks = filter(lambda x: x.no_losses >max_age, tracker_list)  
                    for trk in deleted_tracks:
                        track_id_list.append(trk.id)

                    tracker_list = [x for x in tracker_list if x.no_losses<=max_age]
                    print(tracker_list)
                    BGR=False
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        print(xywh)
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        coord = torch.tensor(xyxy).view(-1, 4)
                        b = xyxy2xywh(coord)  # boxes
                        b[:, 2:] = b[:, 2:] * 1.02 + 10  # box wh * gain + pad
                        coord = xywh2xyxy(b).long()
                        print(coord)
                        clip_boxes(coord, im0.shape)
                        crop = im0[int(coord[0, 1]):int(coord[0, 3]), int(coord[0, 0]):int(coord[0, 2]), ::(1 if BGR else -1)]
                        crop=Image.fromarray(crop)
                        crop.save('temp/'+str(xyxy)+'.jpg')
                        c_image='temp/'+str(xyxy)+'.jpg'
                        #j+=1
                        #print(source.shape)
                        depth=predict_level(c_image, source)
                        if depth==1:
                            sever="Low"
                        elif depth==2:
                            sever="Medium"
                        else:
                            sever="High"
                            
                        print(sever)
                        label = None if hide_labels else (names[c] if hide_conf else f'{"id:",trk.id}{names[c]} {conf:.2f} {sever}')
                        print(label)
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)
                    
                    
                    
            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == 'Linux' and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    w,h,ch=im0.shape
                    print( w,h,ch)
                    font=cv2.FONT_HERSHEY_SIMPLEX
                    fontScale=1
                    color=(0, 0, 255)
                    thickness=2
                    yn=h-(h/2)
                    print(yn)
                    cv2.putText(im0,"Pothole Count:-"+str(len(det)),(0,int(yn)),font, fontScale, 
                 color, thickness, cv2.LINE_AA)
                    print(len(det))
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        print('hi')
                        
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                            print("detect",len(det))
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                            #print(w,h)
                            yn=h-(h/2)
                            font=cv2.FONT_HERSHEY_SIMPLEX
                            fontScale=1
                            color=(0, 0, 255)
                            thickness=2
                            #cv2.putText(vid_cap,"Pothole Count:-"+str(len(det)),(0,int(yn)),font, fontScale, 
                        #color, thickness, cv2.LINE_AA)
                            
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            print(w,h)
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                        print(vid_writer[i])
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        #LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")
        b=det[:,:4]
        print(b)
        
        

    # Print results
    t = tuple(x.t / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        #LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)

        #--weights runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/test/images
def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default= '/content/yolov5/runs/train/exp/weights/best.pt', help='model path or triton URL')
    parser.add_argument('--source', type=str, default='/content/drive/MyDrive/test/images/input.mp4', help='file/dir/URL/glob/screen/0(webcam)')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='(optional) dataset.yaml path')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640], help='inference size h,w')
    parser.add_argument('--conf-thres', type=float, default=0.2, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--visualize', action='store_true', help='visualize features')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default=ROOT / 'runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    parser.add_argument('--dnn', action='store_true', help='use OpenCV DNN for ONNX inference')
    parser.add_argument('--vid-stride', type=int, default=1, help='video frame-rate stride')
    opt, unknown = parser.parse_known_args()
    opt.imgsz *= 2 if len(opt.imgsz) == 1 else 1  # expand
    #print_args(vars(opt))
    return opt


def main():
    check_requirements(exclude=('tensorboard', 'thop'))
    run()
if __name__ == "__main__":
    opt = parse_opt()
    main()
    

YOLOv5  v7.0-71-gc442a2e Python-3.7.9 torch-1.10.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients


<VideoCapture 0000020302CF1D90>
frame 1
tracking []
Detection:  tensor([[718.00000, 720.00000, 842.00000, 774.00000,   0.85227,   0.00000]])
x_box:  []
matched: []
unmatched_det: [0]
unmatched_trks: []
A
tensor([[711, 714, 848, 779]])


C:\Users\kacst\anaconda3\envs\DL\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


ok
no black detected
threshold 2700.0
image size 26715
High
('id:', 'A')pothole 0.85 High
hi
<VideoWriter 0000020302CF1BF0>
tensor([[718., 720., 842., 774.]])
<VideoCapture 0000020302CF1D90>
frame 2
tracking []
Detection:  tensor([[6.92000e+02, 7.51000e+02, 8.09000e+02, 8.13000e+02, 8.09878e-01, 0.00000e+00],
        [8.16000e+02, 5.76000e+02, 8.75000e+02, 6.00000e+02, 5.68828e-01, 0.00000e+00],
        [8.02000e+02, 1.00000e+03, 8.32000e+02, 1.02800e+03, 5.41315e-01, 0.00000e+00],
        [1.45100e+03, 7.07000e+02, 1.51400e+03, 7.33000e+02, 4.99806e-01, 0.00000e+00]])
x_box:  []
matched: []
unmatched_det: [0 1 2 3]
unmatched_trks: []
A
B
C
D
[<tracker.Tracker object at 0x0000020302CBC8C8>, <tracker.Tracker object at 0x0000020302CBC808>, <tracker.Tracker object at 0x00000203037C94C8>, <tracker.Tracker object at 0x00000203037C9488>]
tensor([[1445,  701, 1519,  738]])


C:\Users\kacst\anaconda3\envs\DL\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


...

C:\Users\kacst\anaconda3\envs\DL\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


ok
no black detected
threshold 2700.0
image size 11457
Medium
('id:', 'B')pothole 0.53 Medium
tracking []
Detection:  tensor([[4.80000e+02, 7.14000e+02, 5.97000e+02, 7.71000e+02, 6.58982e-01, 0.00000e+00],
        [1.07100e+03, 6.84000e+02, 1.11700e+03, 7.40000e+02, 5.28106e-01, 0.00000e+00]])
x_box:  []
matched: []
unmatched_det: [0 1]
unmatched_trks: []
A
B
[<tracker.Tracker object at 0x00000203175D2988>, <tracker.Tracker object at 0x00000203175D2848>]
tensor([[473, 708, 603, 776]])


C:\Users\kacst\anaconda3\envs\DL\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


ok
no black detected
threshold 2700.0
image size 26520
Medium
('id:', 'B')pothole 0.66 Medium
tensor([[ 480.,  714.,  597.,  771.],
        [1071.,  684., 1117.,  740.]])
<VideoCapture 0000020302CF1D90>
frame 753
tracking []
Detection:  tensor([[4.40000e+02, 7.52000e+02, 5.56000e+02, 8.18000e+02, 7.63911e-01, 0.00000e+00],
        [1.07400e+03, 7.13000e+02, 1.12400e+03, 7.69000e+02, 6.11535e-01, 0.00000e+00],
        [5.13000e+02, 6.85000e+02, 6.21000e+02, 7.39000e+02, 4.06418e-01, 0.00000e+00]])
x_box:  []
matched: []
unmatched_det: [0 1 2]
unmatched_trks: []
A
B
C
[<tracker.Tracker object at 0x0000020339B7EB88>, <tracker.Tracker object at 0x0000020339B7E848>, <tracker.Tracker object at 0x0000020339B7EFC8>]
tensor([[506, 679, 627, 744]])


C:\Users\kacst\anaconda3\envs\DL\lib\site-packages\sklearn\utils\linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [37]:
from keras.models import load_model
from sklearn.cluster import KMeans
import colorsys as cs
import tensorflow as tf
import os
import numpy as np
#from google.colab.patches import cv2_imshow
try:
    from PIL import Image as pil_image
except ImportError:
    pil_image = None
import keras.utils as image
from keras import backend as K
severity_graph = tf.Graph()
severity_sess = tf.compat.v1.Session(graph = severity_graph)

with severity_graph.as_default():
	with severity_sess.as_default():
		severity_model = load_model(r'training_result.h5')
def find_histogram(clt):
	numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
	# bins is the x-axis value
	(hist, _) = np.histogram(clt.labels_, bins = numLabels)
	# convert value to float
	hist = hist.astype("float")

	# value of each column divide by sum of all value
	hist /= hist.sum()

	return hist
def img_to_array(img, dim_ordering='tf'):
    """Converts a PIL Image instance to a Numpy array.
    # Arguments
        img: PIL Image instance.
        dim_ordering: Image data format.
    # Returns
        A 3D Numpy array.
    # Raises
        ValueError: if invalid `img` or `dim_ordering` is passed.
    """
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    if dim_ordering not in {'th', 'tf'}:
        raise ValueError('Unknown dim_ordering: ', dim_ordering)
    # Numpy array x has format (height, width, channel)
    # or (channel, height, width)
    # but original PIL image has format (width, height, channel)
    x = np.asarray(img, dtype=K.floatx())
    if len(x.shape) == 3:
        if dim_ordering == 'th':
            x = x.transpose(2, 0, 1)
    elif len(x.shape) == 2:
        if dim_ordering == 'th':
            x = x.reshape((1, x.shape[0], x.shape[1]))
        else:
            x = x.reshape((x.shape[0], x.shape[1], 1))
    else:
        raise ValueError('Unsupported image shape: ', x.shape)
    return x

def load_img(path, grayscale=False, target_size=None):
    """Loads an image into PIL format.
    # Arguments
        path: Path to image file
        grayscale: Boolean, whether to load the image as grayscale.
        target_size: Either `None` (default to original size)
            or tuple of ints `(img_height, img_width)`.
    # Returns
        A PIL Image instance.
    # Raises
        ImportError: if PIL is not available.
    """
    if pil_image is None:
        raise ImportError('Could not import PIL.Image. '
                          'The use of `array_to_img` requires PIL.')
    img = pil_image.open(path)
    if grayscale:
        img = img.convert('L')
    else:  # Ensure 3 channel even when loaded image is grayscale
        img = img.convert('RGB')
    if target_size:
        img = img.resize((target_size[1], target_size[0]))
    return img
def detect_level_1(crop_image):
    
    test_image = load_img(crop_image,target_size = (64,64))
    # cv2.imwrite('test_image.jpg', test_image)
    test_image = img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis = 0)

    with severity_graph.as_default():
        with severity_sess.as_default():
            result = severity_model.predict(test_image)
    # print(result)
    # training_set.class_indices
    # print(training_set.class_indices)
    # print(result)
    # max_num = np.argmax(result)
    # print('max', max_num)
    if result <= 0.5:
        prediction = 'crack'
        
        return True
    else:
        prediction = 'not crack'
        print('ok')
        return False
def detect_level_4(crop_image):
	image = cv2.resize(crop_image, (300, 300))
	median = cv2.medianBlur(image, 7)
	# cv2.imshow('blur', median)
	lower_range = np.array([0,0,0])
	upper_range = np.array([15,15,15])

	mask = cv2.inRange(median, lower_range, upper_range)
	# cv2.imshow('mask', mask)
	contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	area = 0
	if len(contours) != 0:
		cv2.drawContours(image, contours, -1, 255, 3)
		# cv2.imwrite('contours.jpg', image)
		c = max(contours, key = cv2.contourArea )

		area = cv2.contourArea(c)
		print('area', area)
		x, y, w, h = cv2.boundingRect(c)
		cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
		# cv2.imwrite('contoursrect.jpg', image)
	else:
		print('no black detected')
		area = 0

	cv2.imshow("image", image)
	# cv2.waitKey(0)
	threshold = (300 * 300) * 0.03
	print('threshold', threshold)
	if(area < threshold):
		# print('not level 4')
		return False;
	else:
		# print('level 4')
		return True
def plot_colors2(hist, centroids):
	# print(centroids)
	bar = np.zeros((50, 300, 3), dtype = 'uint8')
	startX = 0

	# print(list(zip(hist, centroids)))
	results = []
	for(percent, color) in zip(hist, centroids):
		results.append([percent, color])
		endX = startX + (percent * 300)
		cv2.rectangle(bar, (int(startX), 0), (int(endX), 50), color.astype('uint8').tolist(), -1) 
		startX = endX

	return bar, results
def verify_pothole(results):
	# 128 128 128
	# print('verifyinnggg')
	# print(results)
	
	final_h = 0.0
	final_s = 0.0
	final_v = 0.0
	depth_score = 0
	for i in results:
		# print('x')
		# print(i)
		area = i[0]
		r_temp = i[1][0] / 255
		g_temp = i[1][1] / 255
		b_temp = i[1][2] / 255
		# print(r_temp, g_temp, b_temp)
		h, s, v = cs.rgb_to_hsv(r_temp, g_temp, b_temp)
		# print('h = %s, s = %s, v = %s'%(h, s, v))
		temp_depth_score = (1 - s) + (1 - v) * 2 
		# print('temp_depth_score', temp_depth_score)
		if(temp_depth_score > depth_score):
			depth_score = temp_depth_score


	# print('xxxxxxxxxxxxxxxxxxxxxxxxxx')
	# print(depth_score)
	
	depth_score /= 3 #normalise
	return depth_score
def predict_level(crop_image, original_image):
    
	# if(trigger == True):
	# 	isLevel1 = detect_level_1(graph, sess, crop_image)
	# 	if(isLevel1 == True):
	# 		prediction = 'Level 1'
	# 		return prediction
	# 	else:
	# 		prediction = 'Nothing detected'
	# 		return prediction

    image = cv2.imread(crop_image)
    original_image = cv2.imread(original_image)
    # cv2.imshow('ori', original_image)
    # cv2.imshow('crop', image)
    # cv2.waitKey(0)
    # detect level 4 ( black )
    isLevel1 = detect_level_1(crop_image)
    if(isLevel1 == True):
        prediction = 1
        return prediction
    isLevel4 = detect_level_4(image.copy())
    if(isLevel4 == True):
        prediction = 4
        return prediction

    n_cluster = 3
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[0] * img.shape[1],3))
    clt = KMeans(n_cluster)
    clt.fit(img)
    hist = find_histogram(clt)
    bar, color = plot_colors2(hist, clt.cluster_centers_)
    # plt.axis('off')
    # plt.imshow(bar)
    # plt.show()
    depth_score = verify_pothole(color)
    # print('depth: ', depth_score)
    print('image size', image.size)
    #print('original size', original_image.size)
    ratio = image.size / 65500.00
    # print('ratio:', ratio)

    final_ratio = ratio + depth_score
    # print('final ratio', final_ratio)
    if(final_ratio <= 1.0):
        prediction = 2
    else:
        prediction = 3
    # print(prediction)
    return prediction



In [36]:
def assign_detections_to_trackers(trackers,detections,iou_thrd=0.1):
    IOU_mat= np.zeros((len(trackers),len(detections)),dtype=np.float32)
    print(IOU_mat)
    for t,trk in enumerate(trackers):
        #trk = convert_to_cv2bbox(trk) 
        for d,det in enumerate(detections):
         #   det = convert_to_cv2bbox(det)
            IOU_mat[t,d] = helpers.box_iou2(trk,det) 
            print("IOU_mat",IOU_mat)
    
    # Produces matches       
    # Solve the maximizing the sum of IOU assignment problem using the
    # Hungarian algorithm (also known as Munkres algorithm)
    
    matched_idx = linear_assignment(-IOU_mat)        

    unmatched_trackers, unmatched_detections = [], []
    for t,trk in enumerate(trackers):
        if(t not in matched_idx[:,0]):
            unmatched_trackers.append(t)

    for d, det in enumerate(detections):
        if(d not in matched_idx[:,1]):
            unmatched_detections.append(d)

    matches = []
   
    # For creating trackers we consider any detection with an 
    # overlap less than iou_thrd to signifiy the existence of 
    # an untracked object
    
    for m in matched_idx:
        if(IOU_mat[m[0],m[1]]<iou_thrd):
            unmatched_trackers.append(m[0])
            unmatched_detections.append(m[1])
        else:
            matches.append(m.reshape(1,2))
    
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0)
    
    return matches, np.array(unmatched_detections), np.array(unmatched_trackers) 